In [1]:
import os
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.externals.joblib import Parallel, parallel_backend
from sklearn.externals.joblib import register_parallel_backend
from sklearn.externals.joblib import delayed
from sklearn.externals.joblib import cpu_count
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from ipyparallel import Client
from ipyparallel.joblib import IPythonParallelBackend
import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import GridSearchCV

rc = Client(profile='pucv-cluster')
dview = rc.load_balanced_view()#rc[:]

/home/srodriguez/miniconda/envs/jupyterhub/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
register_parallel_backend('ipyparallel',
                          lambda : IPythonParallelBackend(view=dview))

In [3]:
digits = load_digits()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    test_size=0.3)

In [5]:
#some parameters to test in parallel
param_space = {
    'C': np.logspace(-6, 6, 20),
    'gamma': np.logspace(-6,1,20)
}

In [6]:
svc_rbf = SVC(kernel='rbf',
              shrinking=False)

In [7]:
search = GridSearchCV(svc_rbf,
                      param_space,
                      return_train_score=True,
                      n_jobs=len(rc))

In [8]:
%%time
with parallel_backend('ipyparallel'):
    search.fit(X_train, y_train)

CPU times: user 7.02 s, sys: 1.18 s, total: 8.2 s
Wall time: 10.1 s


In [9]:
search_serial = GridSearchCV(svc_rbf,
                      param_space,
                      return_train_score=True,
                      n_jobs=-1)


In [10]:
%%time
search_serial.fit(X_train, y_train)

CPU times: user 5.13 s, sys: 544 ms, total: 5.67 s
Wall time: 34.1 s


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None,
                           shrinking=False, tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-06, 4.28133240e-0...
                         'gamma': array([1.00000000e-06, 2.33572147e-06, 5.45559478e-06, 1.27427499e-05,
       2.97635144e-05, 6.95192796e-05, 1.62377674e-04, 3.79269019e-04,
       8.85866790e-04, 2.06913808e-03, 4.83293024e-03, 1.12883789e-02,
       2.63665090e-02, 6.15848211e-02, 1.43844989e-01, 3.35981829e-01,
       7.84759970e-01, 1.83298071e+00, 4.28133240e+00, 1.00000000e+01])},
             pre_dispatch='2*n_jobs', refit=True, return_tr

In [11]:
results = search.cv_results_
results = pd.DataFrame(results)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.194322,0.015963,0.025566,0.002176,1e-06,1e-06,"{'C': 1e-06, 'gamma': 1e-06}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
1,0.191132,0.020273,0.025795,0.003274,1e-06,2.33572e-06,"{'C': 1e-06, 'gamma': 2.335721469090121e-06}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
2,0.204035,0.027581,0.027158,0.004344,1e-06,5.45559e-06,"{'C': 1e-06, 'gamma': 5.455594781168514e-06}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
3,0.223418,0.040565,0.036884,0.014833,1e-06,1.27427e-05,"{'C': 1e-06, 'gamma': 1.274274985703132e-05}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
4,0.237129,0.061181,0.035988,0.010376,1e-06,2.97635e-05,"{'C': 1e-06, 'gamma': 2.9763514416313192e-05}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
5,0.261247,0.026424,0.046625,0.015640,1e-06,6.95193e-05,"{'C': 1e-06, 'gamma': 6.951927961775606e-05}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
6,0.191666,0.034327,0.028142,0.005018,1e-06,0.000162378,"{'C': 1e-06, 'gamma': 0.0001623776739188721}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
7,0.272340,0.035696,0.030656,0.002691,1e-06,0.000379269,"{'C': 1e-06, 'gamma': 0.000379269019073225}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
8,0.240880,0.071065,0.028305,0.004277,1e-06,0.000885867,"{'C': 1e-06, 'gamma': 0.0008858667904100823}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
9,0.213320,0.040306,0.028653,0.003208,1e-06,0.00206914,"{'C': 1e-06, 'gamma': 0.002069138081114788}",0.111111,0.111111,0.111554,...,0.111377,0.000217,150,0.111443,0.111443,0.111332,0.111332,0.111332,0.111376,0.000054
